<a href="https://colab.research.google.com/github/jonathanyin12/Food.AI/blob/master/FoodAI_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food.AI Training Notebook**
Easy calorie tracking using real-time object detection on Android. 

**[GitHub](https://github.com/jonathanyin12/Food.AI)**

### **Instructions**:
* Prior to starting, create a directory in Google Drive called *food_detection*. Add the **[training dataset](https://drive.google.com/file/d/11WC6XPp4kHGN1vEzl_ZRnFla99pxIs33/view)** and **[label_map.pbtxt](https://github.com/jonathanyin12/Food.AI/blob/master/label_map.pbtxt)** to *food_detection*.

* Make sure runtime is set to use GPU acceleration

* After installing the correct version of NumPy, restart the runtime and run all the cells after.




In [ ]:
# Checking gpu
!nvidia-smi

Sun Jun 21 11:57:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **Installing Dependencies**

In [ ]:
# IMPORTANT: make sure to restart runtime after running this cell
!pip install numpy==1.17.4

     |████████████████████████████████| 20.0MB 1.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [ ]:
%tensorflow_version 1.x

!git clone --quiet https://github.com/tensorflow/models.git # download training tools

!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz # download model
!tar xvzf ssd_mobilenet_v2_coco_2018_03_29.tar.gz
 
!git clone --quiet https://github.com/zamblauskas/oidv4-toolkit-tfrecord-generator.git # download tfrecord generator tool

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive pycocotools

TensorFlow 1.x selected.
--2020-06-21 11:58:11--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.203.128, 2607:f8b0:400c:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.203.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M   269MB/s    in 0.7s    

2020-06-21 11:58:11 (269 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2

In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
!python setup.py -q build
!python setup.py -q install

import os
os.environ['PYTHONPATH'] += '/content/models/research/:/content/models/research/slim/'

%cd /content/models/research/slim
!pip install -e .

%cd /content/models/research
!python object_detection/builders/model_builder_test.py

/content/models/research
object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.
zip_safe flag not set; analyzing archive contents...
object_detection.core.__pycache__.preprocessor.cpython-36: module MAY be using inspect.stack
object_detection.utils.__pycache__.autoaugment_utils.cpython-36: module MAY be using inspect.stack
/content/models/research/slim
Obtaining file:///content/models/research/slim
     |████████████████████████████████| 358kB 2.9MB/s 
  Running setup.py develop for slim
/content/models/research


### **Load food dataset and label map from Drive**

In [ ]:
%cd /content/

from google.colab import drive
from zipfile import ZipFile
from shutil import copyfile

drive.mount('/content/gdrive')
copyfile('/content/gdrive/My Drive/food_detection/label_map.pbtxt', 'label_map.pbtxt')

data_path='/content/gdrive/My Drive/food_detection/OIDv4_ToolKit.zip'
with ZipFile(data_path, 'r') as zipObj:
    zipObj.extractall()

/content


### **Convert dataset into TFRecords format**

In [ ]:
!python oidv4-toolkit-tfrecord-generator/generate-tfrecord.py \
    --classes_file=OIDv4_ToolKit/classes.txt \
    --class_descriptions_file=OIDv4_ToolKit/OID/csv_folder/class-descriptions-boxable.csv \
    --annotations_file=OIDv4_ToolKit/OID/csv_folder/train-annotations-bbox.csv \
    --images_dir=OIDv4_ToolKit/OID/Dataset/train \
    --output_file=train.record

!python oidv4-toolkit-tfrecord-generator/generate-tfrecord.py \
    --classes_file=OIDv4_ToolKit/classes.txt \
    --class_descriptions_file=OIDv4_ToolKit/OID/csv_folder/class-descriptions-boxable.csv \
    --annotations_file=OIDv4_ToolKit/OID/csv_folder/validation-annotations-bbox.csv \
    --images_dir=OIDv4_ToolKit/OID/Dataset/validation \
    --output_file=val.record


Classes: {'Bread': 1, 'Pancake': 2, 'Waffle': 3, 'Bagel': 4, 'Muffin': 5, 'Doughnut': 6, 'Hamburger': 7, 'Pizza': 8, 'Sandwich': 9, 'Hot dog': 10, 'French fries': 11, 'Apple': 12, 'Orange': 13, 'Banana': 14, 'Grape': 15}

W0621 12:01:36.732901 140051544106880 module_wrapper.py:139] From oidv4-toolkit-tfrecord-generator/generate-tfrecord.py:32: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

6419 images

W0621 12:01:36.878070 140051544106880 module_wrapper.py:139] From oidv4-toolkit-tfrecord-generator/generate-tfrecord.py:37: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0621 12:01:36.909929 140051544106880 module_wrapper.py:139] From oidv4-toolkit-tfrecord-generator/generate-tfrecord.py:40: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

...............................................................................................................................................

### **Edit model configuration files**

In [ ]:
import re

filename = '/content/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'
with open(filename) as f:
    s = f.read()
with open(filename, 'w') as f:
    s = re.sub('90', '15', s) # change number of classes from 90 to 15
    s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/content/ssd_mobilenet_v2_coco_2018_03_29/model.ckpt', s) # pass location of model checkpoint
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record-\?\?\?\?\?-of-00100', '/content/train.record', s) # pass location of train tfrecod
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record-\?\?\?\?\?-of-00010', '/content/val.record', s) # pass location of validation tfrecord
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', '/content/label_map.pbtxt', s) # pass location of label map

    f.write(s)

### **Train model**

In [ ]:
!python models/research/object_detection/model_main.py \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --model_dir=/content/gdrive/My\ Drive/food_detection/model_checkpoints

### **Export model to frozen TensorFlow Lite graph**

In [ ]:
lst = os.listdir('/content/gdrive/My Drive/food_detection/model_checkpoints')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --trained_checkpoint_prefix=/content/gdrive/My\ Drive/food_detection/model_checkpoints/$last_model \
    --output_directory=/content/gdrive/My\ Drive/food_detection/model_checkpoints/tflite_model \
    --add_postprocessing_op=true

### **Convert the frozen graph to a TFLite model**

In [ ]:
%cd /content/gdrive/My\ Drive/food_detection/model_checkpoints/tflite_model

!tflite_convert \
    --output_file=food_detect.tflite \
    --graph_def_file=tflite_graph.pb \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'\
    --input_shape=1,300,300,3 \
    --allow_custom_ops